In [1]:
import pandas as pd

# Load each CSV file into pandas DataFrames
order_list = pd.read_csv('preprocessed_OrderList.csv')
freight_rates = pd.read_csv('preprocessed_FreightRates.csv')
wh_costs = pd.read_csv('preprocessed_WhCosts.csv')
wh_capacities = pd.read_csv('preprocessed_WhCapacities.csv')
products_per_plant = pd.read_csv('preprocessed_ProductsPerPlant.csv')
vmi_customers = pd.read_csv('preprocessed_VmiCustomers.csv')
plant_ports = pd.read_csv('preprocessed_PlantPorts.csv')


In [2]:
# Displaying the first few rows and columns of each DataFrame
print("Order List:")
print(order_list.head())

print("\nFreight Rates:")
print(freight_rates.head())

print("\nWarehouse Costs:")
print(wh_costs.head())

print("\nWarehouse Capacities:")
print(wh_capacities.head())

print("\nProducts per Plant:")
print(products_per_plant.head())

print("\nVMI Customers:")
print(vmi_customers.head())

print("\nPlant Ports:")
print(plant_ports.head())


Order List:
       Order ID  Order Date Origin Port Carrier  TPT Service Level  \
0  1.447296e+09  2013-05-26      PORT09   V44_3    1           CRF   
1  1.447158e+09  2013-05-26      PORT09   V44_3    1           CRF   
2  1.447139e+09  2013-05-26      PORT09   V44_3    1           CRF   
3  1.447364e+09  2013-05-26      PORT09   V44_3    1           CRF   
4  1.447364e+09  2013-05-26      PORT09   V44_3    1           CRF   

   Ship ahead day count  Ship Late Day count   Customer  Product ID  \
0                     3                    0  V55555_53     1700106   
1                     3                    0  V55555_53     1700106   
2                     3                    0  V55555_53     1700106   
3                     3                    0  V55555_53     1700106   
4                     3                    0  V55555_53     1700106   

  Plant Code Destination Port  Unit quantity  Weight  
0    PLANT16           PORT09            808   14.30  
1    PLANT16           PORT09 

In [3]:
# Merge tables step-by-step
merged_data = pd.merge(order_list, products_per_plant, on=['Plant Code', 'Product ID'], how='left')
merged_data = pd.merge(merged_data, plant_ports, on='Plant Code', how='left')
merged_data = pd.merge(merged_data, freight_rates, left_on=['Carrier', 'Origin Port', 'Destination Port'],
                       right_on=['Carrier', 'orig_port_cd', 'dest_port_cd'], how='left')
merged_data = pd.merge(merged_data, wh_costs, left_on='Plant Code', right_on='WH', how='left')
merged_data = pd.merge(merged_data, wh_capacities, left_on='Plant Code', right_on='Plant ID', how='left')
merged_data = pd.merge(merged_data, vmi_customers, left_on=['Plant Code', 'Customer'], right_on=['Plant Code', 'Customers'], how='left')


In [4]:
# Display the merged dataset
print("\nMerged Dataset:")
print(merged_data.head())


Merged Dataset:
       Order ID  Order Date Origin Port Carrier  TPT Service Level  \
0  1.447296e+09  2013-05-26      PORT09   V44_3    1           CRF   
1  1.447158e+09  2013-05-26      PORT09   V44_3    1           CRF   
2  1.447139e+09  2013-05-26      PORT09   V44_3    1           CRF   
3  1.447364e+09  2013-05-26      PORT09   V44_3    1           CRF   
4  1.447364e+09  2013-05-26      PORT09   V44_3    1           CRF   

   Ship ahead day count  Ship Late Day count   Customer  Product ID  ...  \
0                     3                    0  V55555_53     1700106  ...   
1                     3                    0  V55555_53     1700106  ...   
2                     3                    0  V55555_53     1700106  ...   
3                     3                    0  V55555_53     1700106  ...   
4                     3                    0  V55555_53     1700106  ...   

  minimum cost rate  mode_dsc  tpt_day_cnt Carrier type       WH Cost/unit  \
0          NaN  NaN       N

In [5]:
#pip install pulp


In [6]:
import pandas as pd
import pulp

In [7]:
merged_data = pd.merge(order_list, freight_rates, left_on=['Carrier', 'Origin Port', 'Destination Port'], right_on=['Carrier', 'orig_port_cd', 'dest_port_cd'], how='left')
merged_data = pd.merge(merged_data, wh_costs, left_on='Plant Code', right_on='WH', how='left')
merged_data = pd.merge(merged_data, wh_capacities, left_on='Plant Code', right_on='Plant ID', how='left')
merged_data = pd.merge(merged_data, products_per_plant, on=['Plant Code', 'Product ID'], how='left')
merged_data = pd.merge(merged_data, vmi_customers.rename(columns={"Customers": "Customer"}), on=['Plant Code', 'Customer'], how='left')
merged_data = pd.merge(merged_data, plant_ports, on='Plant Code', how='left')

In [8]:
merged_data = pd.DataFrame({
    'Order ID': ['Order1', 'Order2', 'Order3'],
    'Product ID': ['Product1', 'Product2', 'Product1'],
    'Plant Code': ['Plant1', 'Plant2', 'Plant1'],
    'Unit quantity': [100, 150, 80],
    'Weight': [200, 300, 150]
})

In [12]:
# Define products, plants, and routes
products = merged_data['Product ID'].unique().tolist()
plants = merged_data['Plant Code'].unique().tolist()
routes = ['Route1', 'Route2', 'Route3']

In [13]:
# Create the LP problem instance
prob = pulp.LpProblem("Route_Optimization", pulp.LpMinimize)


In [14]:
# Decision Variables
x = pulp.LpVariable.dicts("route", ((p, pl, r) for p in products for pl in plants for r in routes), lowBound=0, cat='Continuous')


In [15]:
# Define cost, supply capacity, and product demand dictionaries
# Example: Adjust these with actual data
cost = {(p, pl, r): 10 for p in products for pl in plants for r in routes}
plant_capacity = {'Plant1': 500, 'Plant2': 700}
product_demand = {'Product1': 250, 'Product2': 200}

In [16]:
# Objective Function: Minimize total cost
prob += pulp.lpSum(cost[p, pl, r] * x[p, pl, r] for p in products for pl in plants for r in routes), "Total_Cost"


In [17]:
# Constraints
for pl in plants:
    prob += pulp.lpSum(x[p, pl, r] for p in products for r in routes) <= plant_capacity[pl], f"Supply_{pl}"

for p in products:
    prob += pulp.lpSum(x[p, pl, r] for pl in plants for r in routes) >= product_demand[p], f"Demand_{p}"


In [18]:
# Solve the problem
prob.solve()


1

In [19]:
# Print the status of the solution
print(f"Status: {pulp.LpStatus[prob.status]}")


Status: Optimal


In [20]:
# Access and print variable values
for p in products:
    for pl in plants:
        for r in routes:
            if pulp.value(x[p, pl, r]) > 0:
                print(f"Route {r} for product {p} at plant {pl}: {pulp.value(x[p, pl, r])}")


Route Route1 for product Product1 at plant Plant2: 250.0
Route Route1 for product Product2 at plant Plant2: 200.0


In [21]:
# Optionally, print the complete problem formulation
print("\nProblem formulation:")
print(prob)



Problem formulation:
Route_Optimization:
MINIMIZE
10*route_('Product1',_'Plant1',_'Route1') + 10*route_('Product1',_'Plant1',_'Route2') + 10*route_('Product1',_'Plant1',_'Route3') + 10*route_('Product1',_'Plant2',_'Route1') + 10*route_('Product1',_'Plant2',_'Route2') + 10*route_('Product1',_'Plant2',_'Route3') + 10*route_('Product2',_'Plant1',_'Route1') + 10*route_('Product2',_'Plant1',_'Route2') + 10*route_('Product2',_'Plant1',_'Route3') + 10*route_('Product2',_'Plant2',_'Route1') + 10*route_('Product2',_'Plant2',_'Route2') + 10*route_('Product2',_'Plant2',_'Route3') + 0
SUBJECT TO
Supply_Plant1: route_('Product1',_'Plant1',_'Route1')
 + route_('Product1',_'Plant1',_'Route2')
 + route_('Product1',_'Plant1',_'Route3')
 + route_('Product2',_'Plant1',_'Route1')
 + route_('Product2',_'Plant1',_'Route2')
 + route_('Product2',_'Plant1',_'Route3') <= 500

Supply_Plant2: route_('Product1',_'Plant2',_'Route1')
 + route_('Product1',_'Plant2',_'Route2')
 + route_('Product1',_'Plant2',_'Route3'

In [22]:
# Save the model using pickle
import pickle


In [23]:
# Save the LP problem instance
with open('route_optimization_model.pkl', 'wb') as f:
    pickle.dump(prob, f)

In [1]:
import pulp

In [2]:
# Define the products, plants, and routes
products = ['Product1', 'Product2']
plants = ['Plant1', 'Plant2']
routes = ['Route1', 'Route2', 'Route3']

In [3]:
# Create the LP problem
prob = pulp.LpProblem("Route_Optimization", pulp.LpMinimize)


In [4]:
# Create decision variables
x = pulp.LpVariable.dicts("route", ((p, pl, r) for p in products for pl in plants for r in routes), lowBound=0, cat='Continuous')


In [5]:
# Define costs (for simplicity, using a constant cost of 10 for all routes)
cost = { (p, pl, r): 10 for p in products for pl in plants for r in routes }

In [6]:
# Add objective function
prob += pulp.lpSum(cost[p, pl, r] * x[p, pl, r] for p in products for pl in plants for r in routes)


In [7]:
# Add supply constraints
prob += pulp.lpSum(x['Product1', 'Plant1', r] for r in routes) + pulp.lpSum(x['Product2', 'Plant1', r] for r in routes) <= 500, "Supply_Plant1"
prob += pulp.lpSum(x['Product1', 'Plant2', r] for r in routes) + pulp.lpSum(x['Product2', 'Plant2', r] for r in routes) <= 700, "Supply_Plant2"


In [8]:
# Add demand constraints
prob += pulp.lpSum(x['Product1', pl, r] for pl in plants for r in routes) >= 250, "Demand_Product1"
prob += pulp.lpSum(x['Product2', pl, r] for pl in plants for r in routes) >= 200, "Demand_Product2"


In [9]:
# Solve the problem
prob.solve()

1

In [10]:
# Print the status of the solution
print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


In [11]:
# Print the optimal solution
for v in prob.variables():
    print(v.name, "=", v.varValue)

route_('Product1',_'Plant1',_'Route1') = 0.0
route_('Product1',_'Plant1',_'Route2') = 0.0
route_('Product1',_'Plant1',_'Route3') = 0.0
route_('Product1',_'Plant2',_'Route1') = 250.0
route_('Product1',_'Plant2',_'Route2') = 0.0
route_('Product1',_'Plant2',_'Route3') = 0.0
route_('Product2',_'Plant1',_'Route1') = 0.0
route_('Product2',_'Plant1',_'Route2') = 0.0
route_('Product2',_'Plant1',_'Route3') = 0.0
route_('Product2',_'Plant2',_'Route1') = 200.0
route_('Product2',_'Plant2',_'Route2') = 0.0
route_('Product2',_'Plant2',_'Route3') = 0.0


In [12]:
# Print the optimized objective function value
print("Total Cost =", pulp.value(prob.objective))

Total Cost = 4500.0


In [13]:
import json

# Extract the optimal solution
solution = {v.name: v.varValue for v in prob.variables()}
solution['Total Cost'] = pulp.value(prob.objective)

# Save to a JSON file
with open('model_solution.json', 'w') as f:
    json.dump(solution, f)
